In [0]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [0]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution

In [0]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_prevencao_acidente_trab',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_prevencao_acidente_trab",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [0]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[5]: '/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab/'

In [0]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[6]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab'

In [0]:
azure_path = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'
outputh_path = '{uri}/uds/uniepro/API_InGuru/inguro_vig_acidente_trab'.format(uri=azure_path)

def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [0]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
    data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [0]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [0]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
    if not total_news:
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
              print('Parcial de notícias: ', parcial_news)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

          print("fazendo requisicao")
          print('Data inicial: ', parcial_data_inicial)
          print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)

### Ingestão de anos anteriores

Teve de ser criado pois a API não carrega todos os anos por alguma limitação.

PRECISA SER RODADO APENAS NA PRIMEIRA VEZ

In [0]:
def insere_news(data_inicial, data_final, id, path):
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

  array_dfs = []

  getted_news = 0
    
  total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
  if total_news:
      
    

    parcial_data_inicial = data_inicial
    parcial_data_final = data_final

    while getted_news < total_news:

      parcial_news = total_news

      while parcial_news > 10000:
#         print('Parcial de notícias: ', parcial_news)
        parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
        parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

#       print("fazendo requisicao")
#       print('Data inicial: ', parcial_data_inicial)
#       print('Data final: ', parcial_data_final)
      news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

      #CRIA UM DATAFRAME COM AS NOTICIAS
      noticias = pd.DataFrame(news)

      #corrige as datas
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final

      getted_news += parcial_news

      dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

      schema = StructType([
        StructField("id", LongType(), nullable=True),
        StructField("domain", StringType(), nullable=True),
        StructField("title", StringType(), nullable=True),
        StructField("subtitle", StringType(), nullable=True),
        StructField("author", StringType(), nullable=True),
        StructField("content", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("source", StringType(), nullable=True),
        StructField("source_country", StringType(), nullable=True),
        StructField("source_state", StringType(), nullable=True),
        StructField("crawled_date", StringType(), nullable=True),
        StructField("published_date", StringType(), nullable=True)
      ])

      saveDF = spark.createDataFrame(noticias, schema)
      saveDF = saveDF.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string"))
      saveDF.write.parquet(path, "append")
      print("Total:", total_news)
      print("Inseridas:", getted_news)
    
  else:
    print("Sem dados")


In [0]:
graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '929' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

data_inicial = '01/01/2015 00:00'
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

for i, id in enumerate(graph_ids):
  path = source_lnd + "_" + str(graph_names[i])
  insere_news(data_inicial, data_final, id, path)


Total: 673827
Inseridas: 456
Total: 673827
Inseridas: 8547
Total: 673827
Inseridas: 17478
Total: 673827
Inseridas: 22107
Total: 673827
Inseridas: 23172
Total: 673827
Inseridas: 23721
Total: 673827
Inseridas: 25754
Total: 673827
Inseridas: 35135
Total: 673827
Inseridas: 45074
Total: 673827
Inseridas: 54874
Total: 673827
Inseridas: 64387
Total: 673827
Inseridas: 69061
Total: 673827
Inseridas: 74819
Total: 673827
Inseridas: 84543
Total: 673827
Inseridas: 93838
Total: 673827
Inseridas: 100292
Total: 673827
Inseridas: 106552
Total: 673827
Inseridas: 112139
Total: 673827
Inseridas: 120762
Total: 673827
Inseridas: 129437
Total: 673827
Inseridas: 138818
Total: 673827
Inseridas: 145045
Total: 673827
Inseridas: 153552
Total: 673827
Inseridas: 160371
Total: 673827
Inseridas: 170196
Total: 673827
Inseridas: 179508
Total: 673827
Inseridas: 189199
Total: 673827
Inseridas: 197753
Total: 673827
Inseridas: 206578
Total: 673827
Inseridas: 213267
Total: 673827
Inseridas: 221606
Total: 673827
Inseridas: 230665
Total: 673827
Inseridas: 237744
Total: 673827
Inseridas: 243035
Total: 673827
Inseridas: 251302
Total: 673827
Inseridas: 257203
Total: 673827
Inseridas: 263916
Total: 673827
Inseridas: 270118
Total: 673827
Inseridas: 276598
Total: 673827
Inseridas: 283106
Total: 673827
Inseridas: 288634
Total: 673827
Inseridas: 296016
Total: 673827
Inseridas: 302732
Total: 673827
Inseridas: 309778
Total: 673827
Inseridas: 315912
Total: 673827
Inseridas: 322494
Total: 673827
Inseridas: 329631
Total: 673827
Inseridas: 335567
Total: 673827
Inseridas: 341438
Total: 673827
Inseridas: 351369
Total: 673827
Inseridas: 360704
Total: 673827
Inseridas: 370110
Total: 673827
Inseridas: 379727
Total: 673827
Inseridas: 388036
Total: 673827
Inseridas: 394880
Total: 673827
Inseridas: 402087
Total: 673827
Inseridas: 409409
Total: 673827
Inseridas: 415736
Total: 673827
Inseridas: 423504
Total: 673827
Inseridas: 429506
Total: 673827
Inseridas: 436942
Total: 673827
Inseridas: 442411
Total: 673827
Inseridas: 448306
Total: 673827
Inseridas: 453052
Total: 673827
Inseridas: 458645
Total: 673827
Inseridas: 464949
Total: 673827
Inseridas: 471267
Total: 673827
Inseridas: 477659
Total: 673827
Inseridas: 485653
Total: 673827
Inseridas: 493846
Total: 673827
Inseridas: 502057
Total: 673827
Inseridas: 508616
Total: 673827
Inseridas: 516492
Total: 673827
Inseridas: 523499
Total: 673827
Inseridas: 528444
Total: 673827
Inseridas: 535666
Total: 673827
Inseridas: 540775
Total: 673827
Inseridas: 548979
Total: 673827
Inseridas: 552722
Total: 673827
Inseridas: 560015
Total: 673827
Inseridas: 566643
Total: 673827
Inseridas: 574355
Total: 673827
Inseridas: 583981
Total: 673827
Inseridas: 590935
Total: 673827
Inseridas: 597036
Total: 673827
Inseridas: 606933
Total: 673827
Inseridas: 614752
Total: 673827
Inseridas: 623694
Total: 673827
Inseridas: 630978
Total: 673827
Inseridas: 637085
Total: 673827
Inseridas: 646315
Total: 673827
Inseridas: 653328
Total: 673827
Inseridas: 660695
Total: 673827
Inseridas: 668393
Total: 673827
Inseridas: 673839
Total: 5931
Inseridas: 5931

### Rotina para últimos 7 dias

In [0]:
#SALVANDO EM: /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NOME_DO_NO
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NODO_RAIZ
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_Judicializados
# /tmp/dev/uld/API_I

graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '929' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

#Obtém as notícias de cada nó encontrado começando pelo nó principal.
#Cria um df com as notícias obtidas e salva em um array de dfs.
end_date = datetime.now().strftime("%d/%m/%Y %H:%M")
start_date = (datetime.now() - timedelta(days=7)).strftime("%d/%m/%Y %H:%M")
for id in graph_ids:
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)
  temp_df = get_news(start_date,end_date, id).fillna("Não informado")
  graph_dfs.append(spark.createDataFrame(temp_df))


Total de notícias: 19357
Parcial de notícias: 19357
Parcial de notícias: 11289
fazendo requisicao
Data inicial: 16/02/2023 02:04
Data final: 18/02/2023 23:59
Parcial de notícias: 19357
fazendo requisicao
Data inicial: 18/02/2023 23:59
Data final: 21/02/2023 23:59
Parcial de notícias: 19357
fazendo requisicao
Data inicial: 21/02/2023 23:59
Data final: 22/02/2023 23:59
Parcial de notícias: 19357
fazendo requisicao
Data inicial: 22/02/2023 23:59
Data final: 23/02/2023 23:59
Total de notícias: 89
fazendo requisicao
Data inicial: 16/02/2023 02:04
Data final: 23/02/2023 02:04

In [0]:
#Inserção de hora nas tabelas.
dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0] #Obtendo hora do sistema

for i in range(len(graph_dfs)):
  graph_dfs[i] = graph_dfs[i].withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string")) #Gravando hora em cada um dos dataframes

In [0]:
#Adição de novos registros

for i in range(len(graph_dfs)):
  path = source_lnd + "_" + str(graph_names[i])
  old_df = load_big_df(path)
#   old_df = spark.read.parquet(path)
  max_date = old_df.select(f.max("crawled_date")).collect()[0][0]
  
  min_date = max_date - timedelta(days=7)
  print(max_date, min_date)
  old_df.filter(old_df["crawled_date"] > min_date)
  
  diff_df = graph_dfs[i].join(old_df, on='id', how='left_anti')
  
  diff_df.write.parquet(path, 'append')

2023-02-21 21:38:53 2023-02-14 21:38:53
2023-02-21 18:26:50 2023-02-14 18:26:50

In [0]:
# CARREGAMENTO DAS TABELAS PARA VER SE FORAM SALVAS CORRETAMENTE
test_dfs = []
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  test_dfs.append(spark.read.parquet(path))
  print(path)

[REDACTED]/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab_Nodo_Raiz
[REDACTED]/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab_ACIDENTES_DE_TRABALHO

In [0]:
print(test_dfs[1].count())

5953

In [0]:
# #ZERAR A BASE

# dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

# data = {
#   'id': 0,
#   'domain': 'Não informado',
#   'title': 'Não informado',
#   'subtitle': 'Não informado',
#   'author': 'Não informado',
#   'content': 'Não informado',
#   'url': 'Não informado',
#   'source': 'Não informado',
#   'source_country': 'Não informado',
#   'source_state': 'Não informado',
#   'crawled_date': '1900-01-01 00:00:00',
#   'published_date': '1900-01-01 00:00:00',
#   'dh_insertion_raw': '2020-06-08T01:42:41.000+0000'
# }
# schema = StructType([
#   StructField("id", LongType(), nullable=True),
#   StructField("domain", StringType(), nullable=True),
#   StructField("title", StringType(), nullable=True),
#   StructField("subtitle", StringType(), nullable=True),
#   StructField("author", StringType(), nullable=True),
#   StructField("content", StringType(), nullable=True),
#   StructField("url", StringType(), nullable=True),
#   StructField("source", StringType(), nullable=True),
#   StructField("source_country", StringType(), nullable=True),
#   StructField("source_state", StringType(), nullable=True),
#   StructField("crawled_date", StringType(), nullable=True),
#   StructField("published_date", StringType(), nullable=True),
#   StructField("dh_insertion_raw", StringType(), nullable=True)
#   ])

# data['id'] = int(data['id'])

# # dfff = pd.DataFrame(data, index=[0])
# spark_dfff = spark.createDataFrame([tuple(data.values())], schema=schema)
# for i, df in enumerate(graph_dfs):
#   path = source_lnd + "_" + str(graph_names[i])
#   spark_dfff.write.parquet(path, mode="overwrite")
  

--- Fim Leno ----

In [0]:
data_you_need=pd.DataFrame()

#Obtém as notícias de cada nó encontrado começando pelo nó principal.
#Cria um df com as notícias obtidas e salva em um array de dfs.
for id in graph_ids:
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)
  temp_df = pd.DataFrame.from_dict(API_INGURU(url = url)['data']).fillna("Não informado")
  #temp_df
  data_you_need=data_you_need.append(temp_df,ignore_index=True)
  #graph_dfs.append(spark.createDataFrame(temp_df))
  #temp_df.append(temp_df)
  
  
  
  #data_you_need=pd.DataFrame()
#for infile in glob.glob("*.xlsx"):
    #data = pandas.read_excel(infile)
    #data_you_need=data_you_need.append(data,ignore_index=True)

In [0]:
data_you_need

Out[12]:

,id,domain,title,subtitle,author,content,url,source,source_country,source_state,crawled_date,published_date
0,185445173,www.novanoticias.com.br,Deputado Roberto Hashioka pede duplicação de 3...,"O parlamentar destacou ainda que, desde a impl...",Não informado,"Na sessão plenária desta quinta-feira, 16, o d...",https://www.novanoticias.com.br/noticias/polit...,Nova Notícias,br,Mato Grosso do Sul,2023-02-16 11:20:55.676827,2023-02-17 14:21:46
1,185316683,www.correiodopovo.com.br,Freeway deve receber mais de meio milhão de ve...,Movimento nas estradas estaduais também será i...,Christian Bueller,Segundo a previsão da concessionária da BR-290...,https://www.correiodopovo.com.br/not%C3%ADcias...,Correio do Povo,br,Rio Grande do Sul,2023-02-15 18:15:14,2023-02-17 07:00:00
2,185351987,gazetadealagoas.com.br,,,Não informado,"GAZETA DE ALAGOAS QUINTA-FEIRA, 16 DE FEVEREIR...",https://s3.amazonaws.com/static.resources/orig...,Gazeta de Alagoas - AL,br,Alagoas,2023-02-15 23:43:15,2023-02-16 23:43:15
3,185357022,www.portaldecamaqua.com.br,Menino de quatro anos fica ferido em acidente ...,"Acidente envolveu um Toyota Etios, com placas ...",Não informado,"Na tarde desta quarta-feira, dia 15 de feverei...",https://www.portaldecamaqua.com.br/noticias/60...,Portal de Camaquã,br,,2023-02-15 23:35:05,2023-02-16 23:35:05
4,185351322,drive.google.com,,,Não informado,Poliana Moça\n\nTheo oferece\n\nDê mais confia...,https://s3.amazonaws.com/static.resources/orig...,Jornal Guarulhos Hoje - Impresso - Flip,br,São Paulo,2023-02-15 23:01:10,2023-02-16 23:01:10
...,...,...,...,...,...,...,...,...,...,...,...,...
95,184871925,acervo.folha.uol.com.br,,,Não informado,"FOLHA DE S.PAULO * + *\nSEGUNDA-FEIRA, 13 DE F...",https://s3.amazonaws.com/static.resources/orig...,Folha de São Paulo - Impresso - Flip,br,São Paulo,2023-02-12 23:42:28,2023-02-13 23:42:28
96,185011670,mixvale.com.br,Segurados com doenças na coluna podem pedir ap...,,Não informado,Segurados com doenças na coluna podem pedir ap...,https://mixvale.com.br/2023/02/13/segurados-co...,Mix Vale,br,Minas Gerais,2023-02-13 20:59:36,2023-02-13 20:59:36
97,184965334,folhacapital.com.br,Henfil realiza blitz educativa no Hospital e M...,,Não informado,PALMAS\nPublicado em\n13 de fevereiro de 2023\...,https://folhacapital.com.br/palmas/henfil-real...,Folha Capital,br,Tocantins,2023-02-13 15:18:22,2023-02-13 17:46:03
98,184989290,surgiu.com.br,Secretária de Assistência Social e profissiona...,,Não informado,A secretária de Assistência Social de Marianóp...,https://surgiu.com.br/2023/02/13/secretaria-de...,Surgiu,br,São Paulo,2023-02-13 17:45:07,2023-02-13 17:45:07


In [0]:
nodeID = '820'        
urlNodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)        
newsID = API_INGURU(url = urlNodeID)['data'][0]['id']        
time.sleep(5)        
url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(newsID)        
API_INGURU(url = url)        
df = pd.DataFrame.from_dict(API_INGURU(url = url)['data'])        
time.sleep(5)           
df['date'] = pd.to_datetime('today').strftime("%d/%m/%Y")        
parquet_output = tmp + 'inguru' + '.parquet'        
df.to_parquet(parquet_output, index=False)        
print(parquet_output)        
__drop_directory(adl, schema, table)        
time.sleep(5)        
__upload_file(adl, schema, table,year=year,file=parquet_output)

In [0]:
#Inserção de hora nas tabelas.
dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0] #Obtendo hora do sistema

for df in graph_dfs:
  df = df.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("timestamp")) #Gravando hora em cada um dos dataframes

In [0]:
#Realizando a persistência das tabelas.
#Lê se a mesma já existe, caso sim, compara com a nova e adiciona as diferenças.
#Caso não, pula para a gravação das tabelas.
try:
  for i, df in enumerate(graph_dfs):
    path = source_lnd + "_" + str(graph_names[i])
    old_df = spark.read.parquet(path)
    diff_df = df.subtract(old_df)
    df = df.union(diff_df)
except:
  pass

In [0]:
#Gravação das tabelas
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  df.write.parquet(path, mode="overwrite")

In [0]:
# CARREGAMENTO DAS TABELAS PARA VER SE FORAM SALVAS CORRETAMENTE
test_dfs = []
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  test_dfs.append(spark.read.parquet(path))
  print(path)

[REDACTED]/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab_Nodo_Raiz
[REDACTED]/tmp/dev/uld/API_InGuru/inguro_prevencao_acidente_trab_ACIDENTES_DE_TRABALHO